# Preparing `AlchemicalNetwork`s for use with `fah-alchemy`

`fah-alchemy` is a platform for evluating the free energy differences between chemical systems in an alchemical network.
This notebook will illustrate how to build alchemical networks suitable for submission to a deployed `fah-alchemy` instance.

`fah-alchemy` works in terms of `gufe` objects; the `gufe` module defines the data model for `AlchemicalNetwork`s and all objects it is built up from. We'll import the objects we'll use in this tutorial here.

In [17]:
from gufe import AlchemicalNetwork, Transformation, ChemicalSystem
from gufe.components import ProteinComponent, SmallMoleculeComponent, SolventComponent

## Sample network from `openfe-benchmark`

We'll use a sample network in `openfe-benchmark` for demonstration purposes. The sources can be found here: https://github.com/OpenFreeEnergy/openfe-benchmarks

In particular, we'll use the `tyk2` network. We'll extract ligands manually from the ligand SDF, and the protein target from its PDB.

In [14]:
from importlib import resources
from rdkit import Chem

from openfe_benchmarks import tyk2

In [15]:
tyk2_system = tyk2.get_system()
tyk2_system

In [16]:
tyk2_system.connections

[('lig_ejm_31', 'lig_ejm_50'),
 ('lig_ejm_46', 'lig_jmc_23'),
 ('lig_ejm_31', 'lig_ejm_55'),
 ('lig_ejm_31', 'lig_ejm_48'),
 ('lig_ejm_31', 'lig_ejm_54'),
 ('lig_ejm_31', 'lig_ejm_47'),
 ('lig_ejm_31', 'lig_ejm_46'),
 ('lig_ejm_46', 'lig_jmc_27'),
 ('lig_ejm_46', 'lig_jmc_28'),
 ('lig_ejm_42', 'lig_ejm_43'),
 ('lig_ejm_31', 'lig_ejm_42'),
 ('lig_ejm_45', 'lig_ejm_55')]

## Define `ChemicalSystem`s for network nodes

In [13]:
with resources.path('openfe_benchmarks.data',
                    'tyk2_ligands.sdf') as fn:
    ligands_sdf = Chem.SDMolSupplier(str(fn), removeHs=False)
    ligand_components = [SmallMoleculeComponent(sdf) for sdf in ligands_sdf]

NameError: name 'SmallMoleculeComponent' is not defined

### Define `Component`s for a given `ChemicalSystem`

## Define `Transformation`s between `ChemicalSystem`s as network edges

### Define the `Protocol` used for `Transformation` evaluation